In [1]:
import os
import sys

# NOTE: Please change the folder paths to your current setup.
#Windows
if sys.platform.startswith('win'):
    #Where you downloaded the resource bundle
    os.chdir("D:/")
    #Where you installed spark.    
    os.environ['SPARK_HOME'] = 'D:/Spark/spark-2.0.1-bin-hadoop2.7'
os.curdir

'.'

In [2]:
# Create a variable for our root path
SPARK_HOME = os.environ['SPARK_HOME']

#Add the following paths to the system path. Please check your installation
#to make sure that these zip files actually exist. The names might change
#as versions change.
sys.path.insert(0,os.path.join(SPARK_HOME,"python"))
sys.path.insert(0,os.path.join(SPARK_HOME,"python","lib"))
sys.path.insert(0,os.path.join(SPARK_HOME,"python","lib","pyspark.zip"))
sys.path.insert(0,os.path.join(SPARK_HOME,"python","lib","py4j-0.10.3-src.zip"))

In [3]:
from pyspark import SparkConf, SparkContext
from pyspark import SQLContext
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.feature import VectorAssembler
import numpy as np
import pandas as pd

In [4]:
conf = SparkConf().setMaster("local").setAppName("SparkDecisionTree")
sc = SparkContext(conf=conf)
st = SQLContext(sc)

In [5]:
pandas_df = pd.read_csv('D:/SparkPythonDoBigDataAnalytics-Resources/Data/KNNHomeworkData.csv')
s_df = st.createDataFrame(pandas_df)

In [6]:
features = s_df.columns[0:-1]
assembler = VectorAssembler(
    inputCols=features,
    outputCol='features')

In [7]:
assembled_df = assembler.transform(s_df)
assembled_df.show(5)

+--------------------+------------------+------------+--------------------+------------+-------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-----------+-------------------+-------------------+-----------+--------------------+------------+-------------------+--------------------+--------------------+------------+------------+-----------+--------------------+------------+--------------------+--------------------+--------------------+-----+--------------------+
|                  V1|                V2|          V3|                  V4|          V5|                 V6|                  V7|                  V8|                  V9|                 V10|                 V11|        V12|                V13|                V14|        V15|                 V16|         V17|                V18|                 V19|                 V20|         V21|         V22|        V23|                 V24|         V25|                 V2

In [8]:
train, test = assembled_df.randomSplit([0.8, 0.2])

In [9]:
lr = LogisticRegression(maxIter=50).setLabelCol("Class").setFeaturesCol("features")
model = lr.fit(train)

In [10]:
testing_summary = model.evaluate(test)

In [11]:
testing_summary.predictions.select('Class','prediction').show()

+-----+----------+
|Class|prediction|
+-----+----------+
|    1|       1.0|
|    1|       1.0|
|    1|       1.0|
|    1|       1.0|
|    1|       1.0|
|    1|       1.0|
|    1|       1.0|
|    1|       1.0|
|    1|       1.0|
|    1|       1.0|
|    1|       1.0|
|    1|       1.0|
|    1|       1.0|
|    1|       1.0|
|    1|       1.0|
|    1|       1.0|
|    0|       0.0|
|    1|       1.0|
|    1|       1.0|
|    1|       1.0|
+-----+----------+
only showing top 20 rows



In [12]:

pandas_df = pd.read_csv('D:/SparkPythonDoBigDataAnalytics-Resources/Data/KNNHomeworkTestData.csv')
pandas_df['Class']=0
st_df = st.createDataFrame(pandas_df)

In [13]:
featurest = st_df.columns[0:]
assemblert = VectorAssembler(
    inputCols=features,
    outputCol='features')

In [14]:
assembler_dt = assembler.transform(st_df)

In [15]:
test_summary = model.evaluate(assembler_dt)

In [17]:
test_summary.predictions.select('prediction').show()

+----------+
|prediction|
+----------+
|       0.0|
|       0.0|
|       0.0|
|       0.0|
|       0.0|
|       0.0|
|       0.0|
|       0.0|
|       0.0|
|       0.0|
|       1.0|
|       1.0|
|       1.0|
|       1.0|
|       1.0|
|       1.0|
|       1.0|
|       1.0|
|       1.0|
|       1.0|
+----------+
only showing top 20 rows



In [28]:
df_output = test_summary.predictions.toPandas()

In [42]:
df_output.to_csv('D:\Output.csv',sep=',', encoding='utf-8', index=False)